<a href="https://colab.research.google.com/github/Kyveli-tsioli/hello-world/blob/main/Algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
!ls "/content/drive/My Drive"

 2_logistic_regression.pdf
'2ND trial- IRDM CW1 (1).ipynb'
'2ND trial- IRDM CW1.ipynb'
 3_svms.pdf
'5_neural_networks (1).pdf'
 5_neural_networks.pdf
 6_deep_learning.pdf
 7_unsupervised_intro.pdf
 8_em.pdf
 Assignment_2_Autoencoder.ipynb
'cansat χορηγίες.docx'
'(Chapman & Hall_CRC Texts in Statistical Science) Chris Chatfield - The Analysis Of Time Series - An Introduction-Chapman and Hall_CRC (2003).gdoc'
'(Chapman & Hall_CRC Texts in Statistical Science) Chris Chatfield - The Analysis Of Time Series - An Introduction-Chapman and Hall_CRC (2003).pdf'
'Colab Notebooks'
 coursework-five.zip
'CV_Kyveli_Christina_Tsioli (1).pdf'
'CV_Kyveli_Christina_Tsioli - Kyveli Tsioli.pdf'
 CV_Kyveli_Christina_Tsioli.pdf
 dataset.zip
 exams
'Fund Raising'
 GM
'IMPIRIAL REVAMPED-1.gdoc'
'IMPIRIAL REVAMPED-2.gdoc'
'IMPIRIAL REVAMPED-3.gdoc'
 IRDM
'IRDM_CW1_v2 (1).ipynb'
 IRDM_CW1_v2.ipynb
'[JFA18][DE] Παρουσίαση Χαράς στους TLs της (τους αγαπάει περισσότερο απ'\'' ότι εγώ εσάς).pdf'
'[JFA18][FR

In [11]:
!ls "/content/drive/My Drive/GM"

emissionDists.json  exams_.ipynb  exams.txt  transitionDists.json


In [22]:
transition_file="/content/drive/My Drive/GM/transitionDists.json"
emission_file = "/content/drive/My Drive/GM/emissionDists.json"
#transition_file = 'transitionDists.json'
#emission_file = 'emissionDists.json'

In [23]:
import json
with open(transition_file) as f:
    transitionDists = json.load(f)
with open(emission_file) as f:
    emissionDists = json.load(f)

In [ ]:
states = list(transitionDists.keys())
end_tag_dict = {}
for st in states:
    end_tag_dict[st] = np.NINF
end_tag_dict['<DUMMY_END_TAG>'] = np.NINF
transitionDists['<DUMMY_END_TAG>'] = end_tag_dict

In [24]:
def print_dict(d):
    for key in d:
        print(key)
        print(d.get(key))
        print("----")

In [29]:
#print_dict(transitionDists)
print_dict(emissionDists)
#print(len(transitionDists.get('<DUMMY_START_TAG>')))
#print(len(transitionDists))
#print(transitionDists.get('<DUMMY_START_TAG>'))



PROPN
{'Al': -5.142958103498628, 'Zaman': -8.369546048853167, 'Shaikh': -8.089961186634005, 'Abdullah': -9.405637980539943, 'al': -5.942747029872447, 'Ani': -9.405637980539943, 'Qaim': -8.759010815614891, 'DPA': -9.405637980539943, 'Baghdad': -7.094003052025979, 'Ministry': -8.369546048853167, 'Interior': -9.405637980539943, 'MoI': -9.405637980539943, 'Iraq': -4.720984683983014, 'US': -4.4127347316026375, 'FBI': -6.500228345279238, 'J.': -8.369546048853167, 'Edgar': -9.405637980539943, 'Hoover': -9.405637980539943, 'Weathermen': -9.405637980539943, 'Ghazaliyah': -9.405637980539943, 'Jihad': -6.786253416523389, 'Baathists': -8.759010815614891, 'ARVN': -7.540853376296998, 'Vietnam': -6.36111554281652, 'Kurdistan': -9.405637980539943, 'Democratic': -8.759010815614891, 'Party': -7.692659389165002, 'Mosul': -8.369546048853167, 'Ninevah': -9.405637980539943, 'Province': -7.692659389165002, 'Wednesday': -6.500228345279238, 'Suwayrah': -9.405637980539943, 'Kut': -8.759010815614891, 'Baathist':

In [30]:
def inverse_emission(emissionDists,transitionDists):
    states = list(transitionDists.keys())
    ## fid all the unique observation tokens
    new_dict = {}
    for tag_key in emissionDists:
        for token_key in emissionDists.get(tag_key):
            if(new_dict.get(token_key) == None):
                new_dict[token_key] = {}
            new_dict[token_key][tag_key] = emissionDists.get(tag_key).get(token_key)
            
    for token_key in new_dict:
        emission_prob_dict = new_dict.get(token_key)
        for st in states:
            if(emission_prob_dict.get(st) == None):
                emission_prob_dict[st] = -10000000000
            
    return new_dict

In [31]:
def get_emission_vector(transitionDists,inverese_emission_dict,token):
    states = list(transitionDists.keys())
    #print(states)
    emission_dict = inverese_emission_dict.get(token)
    emission_list = []
    for st in states:
        emission_list.append(emission_dict.get(st))
    return np.array(emission_list)

In [32]:
def get_transition_vector(transitionDists,tag,flag):
    
    states = list(transitionDists.keys())
    
    tag_index= None
    for i,st in enumerate(states):
        if(st == tag):
            tag_index = i
                        
    transition_matrix = []
    for st in states:
        trans = transitionDists.get(st)
        transition_list = []
        for s in states:
            transition_list.append(trans.get(s))
        transition_matrix.append(np.array(transition_list))
    
    transition_matrix = np.array(transition_matrix)
        
    if(flag == 'curr_fixed'):
        return transition_matrix[tag_index]
    elif(flag == 'curr_not_fixed'):
        return transition_matrix[:,tag_index]

In [33]:
inverese_emission_dict = inverse_emission(emissionDists,transitionDists)
print(len(inverese_emission_dict))
print(inverese_emission_dict.get('Professor'))
print(len(inverese_emission_dict.get('Professor'))) #p(v=prof|h)

19673
{'PROPN': -8.369546048853167, 'NOUN': -10.384374920258649, '<DUMMY_START_TAG>': -10000000000, 'PUNCT': -10000000000, 'ADJ': -10000000000, 'VERB': -10000000000, 'DET': -10000000000, 'ADP': -10000000000, 'AUX': -10000000000, 'PRON': -10000000000, 'PART': -10000000000, 'SCONJ': -10000000000, 'NUM': -10000000000, 'ADV': -10000000000, 'CCONJ': -10000000000, 'X': -10000000000, 'INTJ': -10000000000, 'SYM': -10000000000}
18


In [34]:
print(print_dict(inverese_emission_dict))

Η έξοδος ροής περικόπηκε στις τελευταίες 5000 γραμμές.
----
3/8/00
{'NUM': -7.575804407629417, '<DUMMY_START_TAG>': -10000000000, 'PROPN': -10000000000, 'PUNCT': -10000000000, 'ADJ': -10000000000, 'NOUN': -10000000000, 'VERB': -10000000000, 'DET': -10000000000, 'ADP': -10000000000, 'AUX': -10000000000, 'PRON': -10000000000, 'PART': -10000000000, 'SCONJ': -10000000000, 'ADV': -10000000000, 'CCONJ': -10000000000, 'X': -10000000000, 'INTJ': -10000000000, 'SYM': -10000000000}
----
3926
{'NUM': -8.22243157255447, '<DUMMY_START_TAG>': -10000000000, 'PROPN': -10000000000, 'PUNCT': -10000000000, 'ADJ': -10000000000, 'NOUN': -10000000000, 'VERB': -10000000000, 'DET': -10000000000, 'ADP': -10000000000, 'AUX': -10000000000, 'PRON': -10000000000, 'PART': -10000000000, 'SCONJ': -10000000000, 'ADV': -10000000000, 'CCONJ': -10000000000, 'X': -10000000000, 'INTJ': -10000000000, 'SYM': -10000000000}
----
03/09/2000
{'NUM': -7.575804407629417, '<DUMMY_START_TAG>': -10000000000, 'PROPN': -10000000000, 'P

In [ ]:
inverese_emission_dict.get('stupid') 

{'ADJ': -7.360017580918386,
 '<DUMMY_START_TAG>': -10000000000,
 'PROPN': -10000000000,
 'PUNCT': -10000000000,
 'NOUN': -10000000000,
 'VERB': -10000000000,
 'DET': -10000000000,
 'ADP': -10000000000,
 'AUX': -10000000000,
 'PRON': -10000000000,
 'PART': -10000000000,
 'SCONJ': -10000000000,
 'NUM': -10000000000,
 'ADV': -10000000000,
 'CCONJ': -10000000000,
 'X': -10000000000,
 'INTJ': -10000000000,
 'SYM': -10000000000,
 '<DUMMY_END_TAG>': -10000000000}

In [ ]:
#print_dict(inverese_emission_dict)

In [ ]:
emission_vector = get_emission_vector(transitionDists,inverese_emission_dict,'Professor')
print(emission_vector)

[-1.00000000e+10 -8.36954605e+00 -1.00000000e+10 -1.00000000e+10
 -1.03843749e+01 -1.00000000e+10 -1.00000000e+10 -1.00000000e+10
 -1.00000000e+10 -1.00000000e+10 -1.00000000e+10 -1.00000000e+10
 -1.00000000e+10 -1.00000000e+10 -1.00000000e+10 -1.00000000e+10
 -1.00000000e+10 -1.00000000e+10 -1.00000000e+10]


In [ ]:
transition_vector = get_transition_vector(transitionDists,'ADJ','curr_fixed')
print(transition_vector)

[-11.73453985  -3.59102244  -1.9946895   -2.95636797  -0.60755631
  -4.66892648  -5.31944289  -2.53819699  -5.66843176  -4.29556826
  -3.4278208   -3.69184035  -4.90907981  -4.31336232  -3.1074166
  -6.9387493   -8.30055264  -6.71726001  -5.51793375]


In [ ]:
#observation = ['stupid','Professor','is','refusing','to','cooperate']
observation = ['Professor','is','refusing','to','cooperate']

In [ ]:
start_prob_vector = []
for i in  range(len(transitionDists)):
    start_prob_vector.append(np.log(1/(len(transitionDists))))
start_prob_vector = np.array(start_prob_vector)
print(start_prob_vector)

[-2.94443898 -2.94443898 -2.94443898 -2.94443898 -2.94443898 -2.94443898
 -2.94443898 -2.94443898 -2.94443898 -2.94443898 -2.94443898 -2.94443898
 -2.94443898 -2.94443898 -2.94443898 -2.94443898 -2.94443898 -2.94443898
 -2.94443898]


In [ ]:
## compute P(h_t|v_1-t)
def forward_algo(observation,inverese_emission_dict,transitionDists,start_prob_vector):
    
    states = transitionDists.keys()
    a_messages = []
    
    for i, obs in enumerate(observation):
        print(obs)
        if(i==0):
            ## log(p(v1|h)) + log(p(h))
            emission_vector = get_emission_vector(transitionDists,inverese_emission_dict,obs)
            a_0 = emission_vector + start_prob_vector
            a_messages.append(a_0)
        else:
            a = np.zeros(len(states))
            for j,st in enumerate(states):
                emission_vector = get_emission_vector(transitionDists,inverese_emission_dict,obs)
                transition_vector = get_transition_vector(transitionDists,st,'curr_fixed')
                a += np.exp(emission_vector + transition_vector + np.array(a_messages[-1][j]))
            a_messages.append(np.log(a))
            
    return a_messages    

In [ ]:
a_messages = forward_algo(observation,inverese_emission_dict,transitionDists,start_prob_vector)

Professor
is
refusing
to
cooperate


/home/apostolos/.local/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in log


In [ ]:
def filtering(a_messages,i):
    numerator = np.exp(a_messages[i])
    denominator = np.sum(numerator,axis = 0)
    ## return both the proba distr and the likelihood
    return numerator/denominator,denominator

In [ ]:
for i in range(len(a_messages)):
    print(filtering(a_messages,i))

(array([0.        , 0.88234524, 0.        , 0.        , 0.11765476,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ]), 1.382802592133662e-05)
(array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 7.75985926e-02, 0.00000000e+00, 0.00000000e+00,
       9.22125467e-01, 7.89913329e-05, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.96949493e-04, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00]), 1.3847315841952405e-07)
(array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.]), 5.970722448914806e-12)
(array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       7.84658169e-05, 1.31187802e-05, 0.00000000e+00, 3.15440433e-01,
       0.00000000e+00, 0.00000000e+00, 6.71576141e-01, 1.21755932e-02,
       0.00000000e+00,

In [ ]:
def reverse(text):
    result = []
    for i in range(len(text),0,-1):
        result.append(text[i-1])
    return result

In [ ]:
obs = reverse(observation)
print(obs)

['cooperate', 'to', 'refusing', 'is', 'Professor']


In [ ]:
def backward_algo(observation,inverese_emission_dict,transitionDists,start_prob_vector):
    
    observation = reverse(observation)

    states = transitionDists.keys()
    b_messages = []
    
    b_0 = np.zeros(len(states))
    b_messages.append(b_0)
    
    for i, obs in enumerate(observation):
        if(i==len(observation)-1):
            break
        b = np.zeros(len(states))
        emission_vector = get_emission_vector(transitionDists,inverese_emission_dict,obs)
        for j,st in enumerate(states):   
            transition_vector = get_transition_vector(transitionDists,st,"curr_not_fixed")
            b += np.exp(emission_vector[j] + transition_vector + np.array(b_messages[-1][j]))
        b_messages.append(np.log(b))
    
    b_messages.reverse()
    
    return b_messages   

In [ ]:
b_messages = backward_algo(observation,inverese_emission_dict,transitionDists,start_prob_vector)

/home/apostolos/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log


In [ ]:
'''for b in b_messages:
    print(np.exp(b))'''

'for b in b_messages:\n    print(np.exp(b))'

In [ ]:
def smoothing(b_messages,a_messages,i):
    num = np.exp(a_messages[i])*np.exp(b_messages[i])
    denom = np.sum(num,axis = 0)
    return num/denom,denom

In [ ]:
for i in range(len(b_messages)):
    print(smoothing(b_messages,a_messages,i))

(array([0.        , 0.86710583, 0.        , 0.        , 0.13289417,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ]), 7.795206617921301e-18)
(array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 3.85403157e-03, 0.00000000e+00, 0.00000000e+00,
       9.95963318e-01, 6.61647121e-05, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.16485651e-04, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00]), 7.795206617921312e-18)
(array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.]), 7.795206617921292e-18)
(array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       9.42513831e-06, 4.42289522e-07, 0.00000000e+00, 4.39129904e-03,
       0.00000000e+00, 0.00000000e+00, 9.91311475e-01, 3.99979535e-03,
       0.00000000e+00, 

In [ ]:
def viterbi(observation,inverese_emission_dict,transitionDists,start_prob_vector):
    
    states = list(transitionDists.keys())
    
    # length of the observation sequence
    T = len(observation)
    # number of states
    K = len(states)
    
    T1 = np.zeros((K,T))
    T2 = np.zeros((K,T))
    
    ## initialise the initial prob of T1
    for i in range(K):
        T1[i,0] = start_prob_vector[i] + get_emission_vector(transitionDists,inverese_emission_dict,observation[0])[i]

    
    for j in range(1,T):
        ## find for every state the optimal way (max) proba to go there (from all the previous states)
        for i in range(K):
            proba_array = np.zeros(K)
            # get the transition to state i from all the k previous states
            trans = get_transition_vector(transitionDists,states[i],"curr_not_fixed")
            for k in range(K):
                transition = trans[k]
                # get the emission form the current state to the observation
                emission = get_emission_vector(transitionDists,inverese_emission_dict,observation[j])[i]
                # multiply the emsiion the transition and the prob of being in the previous k-th state
                proba_array[k] = T1[k][j-1] + transition + emission
            
            ## get the max proba of geting to state i
            T1[i,j] = np.max(proba_array)
            ## store from which previous state did you start to get this transition
            T2[i,j] = np.argmax(proba_array)
      
    max_states = []
    indices = []
    zt = np.argmax(T1[:,-1])

    max_states.append(states[zt])
    indices.append(zt)
    
    for i in range(T-1,0,-1):
        zt = T2[zt,i]
        zt = int(zt)
        indices.append(zt)
        max_states.append(states[zt])
    
    path = reverse(max_states)
    reversed_indices = reverse(indices)
        
    return path, reversed_indices

In [ ]:
viterbi(observation,inverese_emission_dict,transitionDists,start_prob_vector)

(['PROPN', 'AUX', 'VERB', 'PART', 'VERB'], [1, 8, 5, 10, 5])